In [1]:
import os
from typing import Tuple, Callable, Dict, Optional, List

import numpy as np
import pandas as pd
import scipy.sparse as sp
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from Evaluation.Evaluator import EvaluatorHoldout
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample
from Recommenders.SLIM.Cython.SLIM_BPR_Cython import SLIM_BPR_Cython
from HyperparameterTuning.SearchBayesianSkopt import SearchBayesianSkopt
from HyperparameterTuning.SearchAbstractClass import SearchInputRecommenderArgs
from Recommenders.DataIO import DataIO
from skopt.space import Real, Integer, Categorical
from Recommenders.KNN.ItemKNNCFRecommender import ItemKNNCFRecommender
from Recommenders.GraphBased.RP3betaRecommender import RP3betaRecommender
from Recommenders.SLIM.Cython.SLIM_BPR_Cython import SLIM_BPR_Cython
from Recommenders.SLIM.SLIMElasticNetRecommender import SLIMElasticNetRecommender,MultiThreadSLIM_SLIMElasticNetRecommender



columns_name = ['user_id','item_id','impression_list','data']

dataset_l = pd.read_csv('/Users/loren/Documents/GitHub/RecSys/dati/data_ICM_length.csv', sep=',')
dataset_t = pd.read_csv('/Users/loren/Documents/GitHub/RecSys/dati/data_ICM_type.csv', sep=',')
dataset = pd.read_csv('/Users/loren/Documents/GitHub/RecSys/dati/interactions_and_impressions.csv', sep=',')
test_users = pd.read_csv('/Users/loren/Documents/GitHub/RecSys/dati/data_target_users_test.csv',sep = ',')
dataset.columns = columns_name

c:\Users\loren\anaconda3\envs\RecSysFramework\lib\site-packages\IPython\core\interactiveshell.py:3185: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [7]:
from Data_manager.Build_Matrix.Data_import import build_ICM_type,build_ICM_lengh,build_URM_ICM_onlyURM_item
URM_ALL,ICM=build_URM_ICM_onlyURM_item(dataset, dataset_t, dataset_l)
ICM_TYPE   =build_ICM_type(dataset_t)
ICM_LENGHT =build_ICM_lengh(dataset_l)

In [6]:
URM_train_validation, URM_test = split_train_in_two_percentage_global_sample(URM, train_percentage = 0.80)
URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_train_validation, train_percentage = 0.80)

evaluator_validation = EvaluatorHoldout(URM_validation, cutoff_list=[10])
evaluator_test = EvaluatorHoldout(URM_test, cutoff_list=[10])

EvaluatorHoldout: Ignoring 741 ( 1.8%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 329 ( 0.8%) Users that have less than 1 test interactions


In [8]:
import scipy.sparse as sps

stacked_URM_1 = sps.vstack([URM_train, ICM_TYPE.T])
stacked_URM_1_1 = sps.vstack([URM_train, ICM_TYPE.T, ICM_TYPE.T, ICM_TYPE.T, ICM_TYPE.T])

stacked_URM_2 = sps.vstack([URM_train, ICM_LENGHT.T])
stacked_URM_3 = sps.vstack([URM_train, ICM_TYPE.T, ICM_LENGHT.T])

In [18]:
import os

output_folder_path = "result_experiments_SLIMElasticNetRecommender_last_1"

# If directory does not exist, create
if not os.path.exists(output_folder_path):
    os.makedirs(output_folder_path)

n_cases = 500
n_random_starts = int(n_cases*0.4)
metric_to_optimize = "MAP"   
cutoff_to_optimize = 10

hyperparameters_range_dictionary = {
        "topK": Integer(100, 10000),
        "l1_ratio": Real(low=1e-3, high=0.1, prior='log-uniform'),
        "alpha": Real(low=1e-3, high=0.1, prior='uniform'),
        #"mw": Integer(1, 20),
    }

earlystopping_keywargs = {"validation_every_n": 5,
                              "stop_on_validation": True,
                              "evaluator_object": URM_validation,
                              "lower_validations_allowed": 5,
                              "validation_metric": metric_to_optimize,
                              }
recommender_input_args = SearchInputRecommenderArgs(
                CONSTRUCTOR_POSITIONAL_ARGS = [stacked_URM_1_1],
                CONSTRUCTOR_KEYWORD_ARGS = {},
                FIT_POSITIONAL_ARGS = [],
                FIT_KEYWORD_ARGS = {},
                EARLYSTOPPING_KEYWORD_ARGS = {},
            )

recommender_class = SLIMElasticNetRecommender


hyperparameterSearch = SearchBayesianSkopt(recommender_class,
                                        evaluator_validation=evaluator_validation,
                                        evaluator_test=evaluator_test)

In [19]:
hyperparameterSearch.search(recommender_input_args,
                    hyperparameter_search_space = hyperparameters_range_dictionary,
                    n_cases = n_cases,
                    n_random_starts = n_random_starts,
                    save_model = "last",
                    output_folder_path = output_folder_path, # Where to save the results
                    output_file_name_root = recommender_class.RECOMMENDER_NAME, # How to call the files
                    metric_to_optimize = metric_to_optimize,
                    cutoff_to_optimize = cutoff_to_optimize,
                    resume_from_saved = True,
                    recommender_input_args_last_test=recommender_input_args
                    )

SearchBayesianSkopt: Resuming 'SLIMElasticNetRecommender' Failed, no such file exists.

Iteration No: 1 started. Evaluating function at random point.
SearchBayesianSkopt: Testing config: {'topK': 1307, 'l1_ratio': 0.048945223846787185, 'alpha': 0.08860202924149128}
SLIMElasticNetRecommender: Processed 6522 (26.6%) in 5.00 min. Items per second: 21.73
SLIMElasticNetRecommender: Processed 14069 (57.4%) in 10.00 min. Items per second: 23.44
SLIMElasticNetRecommender: Processed 22321 (91.1%) in 15.00 min. Items per second: 24.80
SLIMElasticNetRecommender: Processed 24507 (100.0%) in 16.38 min. Items per second: 24.93
EvaluatorHoldout: Processed 40888 (100.0%) in 32.81 sec. Users per second: 1246
SearchBayesianSkopt: New best config found. Config 0: {'topK': 1307, 'l1_ratio': 0.048945223846787185, 'alpha': 0.08860202924149128} - results: PRECISION: 0.0245035, PRECISION_RECALL_MIN_DEN: 0.0386796, RECALL: 0.0355314, MAP: 0.0114494, MAP_MIN_DEN: 0.0180912, MRR: 0.0894800, NDCG: 0.0386963, F1: 

KeyboardInterrupt: 

In [13]:
from Recommenders.DataIO import DataIO
from Recommenders.SLIM.SLIMElasticNetRecommender import SLIMElasticNetRecommender
import pandas as pd
recommender_class = SLIMElasticNetRecommender

output_folder_path = "result_experiments"
data_loader = DataIO(folder_path = output_folder_path)
search_metadata = data_loader.load_data(recommender_class.RECOMMENDER_NAME + "_metadata.zip")
time_df = search_metadata["time_df"]

print('mean-time:')
print(time_df['train'].mean())


result_on_test_df = search_metadata["result_on_test_df"]
result_best_on_test = search_metadata["result_on_last"]
print('BEST_HYPERPARAMETERS: ')
print(search_metadata["hyperparameters_best"])
print(pd.DataFrame(result_on_test_df))
print(result_best_on_test)

mean-time:
392.422853257921
BEST_HYPERPARAMETERS: 
{'topK': 668, 'l1_ratio': 0.06262713223835095, 'alpha': 0.001}
          PRECISION PRECISION_RECALL_MIN_DEN    RECALL       MAP MAP_MIN_DEN  \
   cutoff                                                                      
0  10      0.048502                 0.067374  0.059069  0.022471    0.030846   
1  10           NaN                      NaN       NaN       NaN         NaN   
2  10           NaN                      NaN       NaN       NaN         NaN   
3  10           NaN                      NaN       NaN       NaN         NaN   
4  10           NaN                      NaN       NaN       NaN         NaN   
...             ...                      ...       ...       ...         ...   
95 10           NaN                      NaN       NaN       NaN         NaN   
96 10           NaN                      NaN       NaN       NaN         NaN   
97 10           NaN                      NaN       NaN       NaN         NaN   
98 10 